In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoMLOps

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/automlops/blob/main/example/automlops_example_notebook.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/automlops/blob/main/example/automlops_example_notebook.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/automlops/main/example/automlops_example_notebook.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

# Overview

In this tutorial, you will build two [Vertex AI](https://cloud.google.com/vertex-ai) pipelines, complete with an integrated CI/CD pipeline. This tutorial will walk you through how to use AutoMLOps to define, create and run pipelines.

# Objective
In this tutorial, you will learn how to create and run MLOps pipelines integrated with CI/CD. This tutorial goes through an example pipeline that is defined two ways: first using a custom python syntax, and second using Kubeflow syntax (either option is valid, whichever is preferred is up to you). The example pipeline builds and deploys a classification model; the pipeline go through a very basic workflow:
1. create_dataset: A custom component that will export the dataset from BQ to GCS as a csv.
2. train_model: A custom component that will train a decision tree classifier on the training data.
3. deploy_model: A custom component that will upload the saved_model to Vertex AI Model Registry and deploy it to an endpoint.

# Prerequisites

In order to use AutoMLOps, the following are required:

- Jupyter (or Jupyter-compatible) notebook environment
- [Notebooks API](https://console.cloud.google.com/marketplace/product/google/notebooks.googleapis.com) enabled
- Python 3.0 - 3.10
- [Google Cloud SDK 407.0.0](https://cloud.google.com/sdk/gcloud/reference)
- [beta 2022.10.21](https://cloud.google.com/sdk/gcloud/reference/beta)
- `git` installed
- `git` logged-in:
```
  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"
```
- [Application Default Credentials (ADC)](https://cloud.google.com/docs/authentication/provide-credentials-adc) are setup. This can be done through the following commands:
```
gcloud auth application-default login
gcloud config set account <account@example.com>
```

# Dependencies
- `autoflake==2.0.0`,
- `docopt==0.6.2`,
- `ipython==7.34.0`,
- `pipreqs==0.4.11`,
- `pyflakes==3.0.1`,
- `PyYAML==5.4.1`,
- `yarg==0.1.9`

# APIs & IAM
AutoMLOps will enable the following APIs:
- [cloudresourcemanager.googleapis.com](https://cloud.google.com/resource-manager/reference/rest)
- [aiplatform.googleapis.com](https://cloud.google.com/vertex-ai/docs/reference/rest)
- [artifactregistry.googleapis.com](https://cloud.google.com/artifact-registry/docs/reference/rest)
- [cloudbuild.googleapis.com](https://cloud.google.com/build/docs/api/reference/rest)
- [cloudscheduler.googleapis.com](https://cloud.google.com/scheduler/docs/reference/rest)
- [cloudtasks.googleapis.com](https://cloud.google.com/tasks/docs/reference/rest)
- [compute.googleapis.com](https://cloud.google.com/compute/docs/reference/rest/v1)
- [iam.googleapis.com](https://cloud.google.com/iam/docs/reference/rest)
- [iamcredentials.googleapis.com](https://cloud.google.com/iam/docs/reference/credentials/rest)
- [ml.googleapis.com](https://cloud.google.com/ai-platform/training/docs/reference/rest)
- [run.googleapis.com](https://cloud.google.com/run/docs/reference/rest)
- [storage.googleapis.com](https://cloud.google.com/storage/docs/apis)
- [sourcerepo.googleapis.com](https://cloud.google.com/source-repositories/docs/reference/rest)

AutoMLOps will update [IAM privileges](https://cloud.google.com/iam/docs/understanding-roles) for the following accounts:
1. Pipeline Runner Service Account (one is created if it does exist, defaults to: vertex-pipelines@PROJECT_ID.iam.gserviceaccount.com). Roles added:
- roles/aiplatform.user
- roles/artifactregistry.reader
- roles/bigquery.user
- roles/bigquery.dataEditor
- roles/iam.serviceAccountUser
- roles/storage.admin
- roles/run.admin
2. Cloudbuild Default Service Account (PROJECT_NUMBER@cloudbuild.gserviceaccount.com). Roles added:
- roles/run.admin
- roles/iam.serviceAccountUser
- roles/cloudtasks.enqueuer
- roles/cloudscheduler.admin

# User Guide

For a user-guide, please view these [slides](../AutoMLOps_Implementation_Guide_External.pdf).

# Costs

This tutorial uses billable components of Google Cloud:
- Vertex AI
- Artifact Registry
- Cloud Storage
- Cloud Source Repository
- Cloud Build
- Cloud Run
- Cloud Scheduler

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

# Ground-rules for using AutoMLOps
1. Do not use globally defined variables within the scope of a custom component. These custom components will become containers and will have no reference to the globals within this notebook.
2. Do not reference functions or variables from another cell within the scope of a custom component.
3. Test each of your components for accuracy and correctness before running them using AutoMLOps. We cannot fix bugs automatically; bugs are much more difficult to fix once they are made into pipelines.
4. If you are using Kubeflow, be sure to define all the requirements needed to run the custom component - it can be easy to leave out packages which will cause the container to fail when running within a pipeline. 


# Dataset
For training data, we are using the [dry beans dataset](https://archive.ics.uci.edu/ml/datasets/dry+bean+dataset) which contains images of seven different types of dry beans taken with a high-resolution camera. The raw dataset can be found [here](https://github.com/GoogleCloudPlatform/automlops/blob/main/example/data/Dry_Beans_Dataset.csv).

# Setup Git
Set up your git configuration below

In [ ]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

# Install AutoMLOps

Clone the repo and install either via setup.py or wheel (wheel requires less processing):
- setup.py: `pip install .`
- wheel: `pip install dist/AutoMLOps-1.0.1-py2.py3-none-any.whl`

In [1]:
!pip3 install ../dist/AutoMLOps-1.0.2-py2.py3-none-any.whl --user

Processing /home/jupyter/test/AutoMLOps-github/dist/AutoMLOps-1.0.1-py2.py3-none-any.whl


# Restart the kernel
Once you've installed the AutoMLOps package, you need to restart the notebook kernel so it can find the package.

**Note: Once this cell has finished running, continue on. You do not need to re-run any of the cells above.**

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

# Set your project ID
Set your project ID below. If you don't know your project ID, leave the field blank and the following cells may be able to find it.

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: automlops-sandbox


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


# Upload Data
This will create a BQ table and upload the Dry Beans csv. 

In [4]:
!python3 -m data.load_data_to_bq --project $PROJECT_ID --file data/Dry_Beans_Dataset.csv

Dataset automlops-sandbox.test_dataset already exists
Table test_dataset.dry-beans already exists


## 1. AutoMLOps Pipeline - Custom Python Syntax
This workflow will generate a pipeline without using Kubeflow spec. `generate()` will create all the necessary files but not run them. `go()` will create all the necessary files, resources, push the code to the source repo to trigger the build, and then submit a Pipeline training job to Vertex AI. Please see the [readme](https://github.com/GoogleCloudPlatform/automlops/blob/main/README.md) for more information.

**Note: You can run this entire workflow below without even having Kubeflow installed - give it a try!**

## Imports
First, import AutoMLOps in a separate cell. Then define imports used for all custom components of the pipeline inside a single cell Place the `%%define_imports` cell magic decorator at the top of your imports cell. This cell magic decorator comes built-in with AutoMLOps.

In [5]:
from AutoMLOps import AutoMLOps

In [6]:
%%define_imports
import json
import pandas as pd
from google.cloud import aiplatform
from google.cloud import aiplatform_v1
from google.cloud import bigquery
from google.cloud import storage
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from joblib import dump
import pickle
import os

## Data Loading
Define a custom component for loading and creating a dataset. Place the `%%define_component` cell magic decorator at the top of your custom component cell. This cell magic decorator comes built-in with AutoMLOps. Define your component by calling `AutoMLOps.makeComponent` at the top of your cell, right after `%%define_component`. Give the name, description, and parameters for your component.

**Note: we currently only support python primitive types for component parameters. If you would like to use something more advanced, please use the Kubeflow spec instead (see below in this notebook).**

In [7]:
%%define_component
AutoMLOps.makeComponent(
    name="create_dataset",
    description="Loads data from BQ and writes a dataframe as a csv to GCS.", # optional
    params=[
        {"name": "bq_table", "type": str}, # descriptions are optional
        {"name": "data_path", "type": str, "description": "GS location where the training data is written."},
        {"name": "project_id", "type": str, "description": "Project_id."}
    ]
)
# Component code goes below:
bq_client = bigquery.Client(project=project_id)

def get_query(bq_input_table: str) -> str:
    """Generates BQ Query to read data.

    Args:
    bq_input_table: The full name of the bq input table to be read into
    the dataframe (e.g. <project>.<dataset>.<table>)
    Returns: A BQ query string.
    """
    return f"""
    SELECT *
    FROM `{bq_input_table}`
    """

def load_bq_data(query: str, client: bigquery.Client) -> pd.DataFrame:
    """Loads data from bq into a Pandas Dataframe for EDA.
    Args:
    query: BQ Query to generate data.
    client: BQ Client used to execute query.
    Returns:
    pd.DataFrame: A dataframe with the requested data.
    """
    df = client.query(query).to_dataframe()
    return df

dataframe = load_bq_data(get_query(bq_table), bq_client)
dataframe.to_csv(data_path)

## Model Training
Define a custom component for training a model.

In [8]:
%%define_component
AutoMLOps.makeComponent(
    name="train_model",
    description="Trains a decision tree on the training data.",
    params=[
        {"name": "model_directory", "type": str, "description": "GS location of saved model."},
        {"name": "data_path", "type": str, "description": "GS location where the training data."}
    ]
)
# Component code goes below:
def save_model(model, model_directory):
    """Saves a model to uri."""
    filename = f'model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    
    bucket_name = model_directory.split('/')[2]
    prefix='/'.join(model_directory.split('/')[3:])
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(os.path.join(prefix, filename))
    blob.upload_from_filename(filename)

df = pd.read_csv(data_path)
labels = df.pop("Class").tolist()
data = df.values.tolist()
x_train, x_test, y_train, y_test = train_test_split(data, labels)
skmodel = DecisionTreeClassifier()
skmodel.fit(x_train,y_train)
score = skmodel.score(x_test,y_test)
print('accuracy is:',score)

output_uri = os.path.join(model_directory, f'model.pkl')
save_model(skmodel, model_directory)

## Uploading & Deploying the Model
Define a custom component for uploading and deploying a model in Vertex AI.

In [9]:
%%define_component
AutoMLOps.makeComponent(
    name="deploy_model",
    description="Trains a decision tree on the training data.",
    params=[
        {"name": "model_directory", "type": str, "description": "GS location of saved model."},
        {"name": "project_id", "type": str, "description": "Project_id."},
        {"name": "region", "type": str, "description": "Region."}
    ]
)
# Component code goes below:
aiplatform.init(project=project_id, location=region)
deployed_model = aiplatform.Model.upload(
    display_name="beans-model-pipeline",
    artifact_uri = model_directory,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
)
endpoint = deployed_model.deploy(machine_type="n1-standard-4")

## Define the Pipeline
Define the pipeline by calling `AutoMLOps.makePipeline`. Give the name, description, and parameters for your pipeline. Define the structure by listing, in order, the components you would like your pipeline to call. Give a tuple that maps the pipeline parameters and the component parameters. 

**Note: we currently only support python primitive types for pipeline parameters. If you would like to use something more advanced, please use the Kubeflow spec instead (see below in this notebook).**

In [10]:
AutoMLOps.makePipeline(
    name="training-pipeline",
    description="description", # optional
    params=[
        {"name": "bq_table", "type": str}, # descriptions are optional
        {"name": "model_directory", "type": str, "description": "Description."},
        {"name": "data_path", "type": str, "description": "Description."},
        {"name": "project_id", "type": str, "description": "Description."},
        {"name": "region", "type": str, "description": "Description."}
    ],
    pipeline=[{
        "component_name": "create_dataset", "param_mapping": [
            ("bq_table", "bq_table"), # (component_param_name, pipeline_param_name)
            ("data_path", "data_path"),
            ("project_id", "project_id")
        ]
    },
    {
        "component_name": "train_model", "param_mapping": [
            ("model_directory", "model_directory"),
            ("data_path", "data_path")
        ]
    },
    {
        "component_name": "deploy_model", "param_mapping": [
            ("model_directory", "model_directory"),
            ("project_id", "project_id"),
            ("region", "region")
        ]
    }]
)

## Define the Pipeline Arguments

In [11]:
pipeline_params = {
    "bq_table": f"{PROJECT_ID}.test_dataset.dry-beans",
    "model_directory": f"gs://{PROJECT_ID}-bucket/trained_models/{datetime.datetime.now()}",
    "data_path": f"gs://{PROJECT_ID}-bucket/data",
    "project_id": f"{PROJECT_ID}",
    "region": "us-central1"
}

## Generate and Run the pipeline
`AutoMLOps.generate` generates the code for the MLOps pipeline. `AutoMLOps.go` generates the code and runs the pipeline.

In [12]:
AutoMLOps.generate(project_id=PROJECT_ID, pipeline_params=pipeline_params, use_kfp_spec=False, run_local=False, schedule_pattern='0 */12 * * *')

INFO: Successfully saved requirements file in AutoMLOps/components/component_base/requirements.txt


In [13]:
AutoMLOps.go(project_id=PROJECT_ID, pipeline_params=pipeline_params, use_kfp_spec=False, run_local=False, schedule_pattern='0 */12 * * *')

INFO: Successfully saved requirements file in AutoMLOps/components/component_base/requirements.txt
 Updating required API services in project automlops-sandbox 
Operation "operations/acat.p2-45373616427-0dd7f42f-11d9-4ca9-9738-98f0e9962d00" finished successfully.
 Checking for Artifact Registry: vertex-mlops-af in project automlops-sandbox 
Listing items under project automlops-sandbox, location us-central1.

vertex-mlops-af  DOCKER  STANDARD_REPOSITORY  Artifact Registry vertex-mlops-af in us-central1.  us-central1          Google-managed key  2023-01-11T22:12:26  2023-03-03T21:49:47  12137.168
Artifact Registry: vertex-mlops-af already exists in project automlops-sandbox
 Checking for GS Bucket: automlops-sandbox-bucket in project automlops-sandbox 
gs://automlops-sandbox-bucket/
GS Bucket: automlops-sandbox-bucket already exists in project automlops-sandbox
 Checking for Service Account: vertex-pipelines in project automlops-sandbox 
Pipeline Runner Service Account         vertex-pi

INFO:root:Pushing code to automlops branch, triggering cloudbuild...
INFO:root:Cloudbuild job running at: https://console.cloud.google.com/cloud-build/builds;region=us-central1
INFO:root:
#################################################################
#                                                               #
#                       RESOURCES MANIFEST                      #
#---------------------------------------------------------------#
#     Generated resources can be found at the following urls    #
#                                                               #
#################################################################

INFO:root:Google Cloud Storage Bucket: https://console.cloud.google.com/storage/automlops-sandbox-bucket
INFO:root:Artifact Registry: https://console.cloud.google.com/artifacts/docker/automlops-sandbox/us-central1/vertex-mlops-af
INFO:root:Service Accounts: https://console.cloud.google.com/iam-admin/serviceaccounts?project=automlops-sandbox
INFO:r

To https://source.developers.google.com/p/automlops-sandbox/r/AutoMLOps-repo
   d217fb9..83b14ed  automlops -> automlops


## Set Pipeline Compute Resources
Use the `custom_training_job_specs` parameter to specify custom resources for any custom component in the pipeline. The example below uses a GPU for accelerated training.
See [Machine types](https://cloud.google.com/vertex-ai/docs/training/configure-compute#machine-types) and [GPUs](https://cloud.google.com/vertex-ai/docs/training/configure-compute#specifying_gpus).

In [14]:
AutoMLOps.generate(project_id=PROJECT_ID, 
                   pipeline_params=pipeline_params, 
                   use_kfp_spec=True, 
                   run_local=False, 
                   schedule_pattern='0 */12 * * *',
                   custom_training_job_specs = [{
                       'component_spec': 'train_model',
                       'display_name': 'train-model-accelerated',
                       'machine_type': 'a2-highgpu-1g',
                       'accelerator_type': 'NVIDIA_TESLA_A100',
                       'accelerator_count': '1'
                   }]
)

## Default Run Settings
Below are the default parameters for running `AutoMLOps`. Note there are only two required parameters:
1. project_id
2. pipeline_params

The other parameters are optional. You can customize the output of `AutoMLOps` by specify the resources you'd like to use (or specifying the name of resources you'd like `AutoMLOps` to create if they don't currently exist). A description of the parameters is below:
- `project_id`: The project ID.
- `pipeline_params`: Dictionary containing runtime pipeline parameters.
- `af_registry_location`: Region of the Artifact Registry.
- `af_registry_name`: Artifact Registry name where components are stored.
- `cb_trigger_location`: The location of the cloudbuild trigger.
- `cb_trigger_name`: The name of the cloudbuild trigger.
- `cloud_run_location`: The location of the cloud runner service.
- `cloud_run_name`: The name of the cloud runner service.
- `cloud_tasks_queue_location`: The location of the cloud tasks queue.
- `cloud_tasks_queue_name`: The name of the cloud tasks queue.
- `csr_branch_name`: The name of the csr branch to push to to trigger cb job.
- `csr_name`: The name of the cloud source repo to use.
- `custom_training_job_specs`: Specifies the specs to run the training job with.
- `gs_bucket_location`: Region of the GS bucket.
- `gs_bucket_name`: GS bucket name where pipeline run metadata is stored.
- `pipeline_runner_sa`: Service Account to runner PipelineJobs.
- `run_local`: Flag that determines whether to use Cloud Run CI/CD.
- `schedule_location`: The location of the scheduler resource.
- `schedule_name`: The name of the scheduler resource.
- `schedule_pattern`: Cron formatted value used to create a Scheduled retrain job.
- `use_kfp_spec`: Flag that specifies whether you are using Kubeflow spec or not.
- `vpc_connector`: The name of the vpc connector to use.

The `run_local` parameter specifies whether to use the generated `scripts/run_all.sh` local script to submit the build job and PipelineJob. If this parameter is set to False, `AutoMLOps` will use the cloud [CI/CD workflow](https://github.com/GoogleCloudPlatform/automlops#cloud-continuous-integration-and-continuous-deployment-workflow). The run above uses `run_local=False`, and the run below uses `run_local=True`, notice the differences in output (`run_local=True` means you will not use Cloud Source Repository to trigger build jobs on push). 

In [18]:
AutoMLOps.go(project_id=PROJECT_ID, # required
             pipeline_params=pipeline_params, # required
             af_registry_location='us-central1', # default
             af_registry_name='vertex-mlops-af', # default
             cb_trigger_location='us-central1', # default
             cb_trigger_name='automlops-trigger', # default
             cloud_run_location='us-central1', # default
             cloud_run_name='run-pipeline', # default
             cloud_tasks_queue_location='us-central1', # default
             cloud_tasks_queue_name='queueing-svc', # default
             csr_branch_name='automlops', # default
             csr_name='AutoMLOps-repo', # default
             custom_training_job_specs=None, # default
             gs_bucket_location='us-central1', # default
             gs_bucket_name=None, # default
             pipeline_runner_sa=None, # default
             run_local=True, # default
             schedule_location='us-central1', # default
             schedule_name='AutoMLOps-schedule', # default
             schedule_pattern='No Schedule Specified', # default
             use_kfp_spec=False, # default
             vpc_connector='No VPC Specified' # default
)

INFO: Successfully saved requirements file in AutoMLOps/components/component_base/requirements.txt
 Updating required API services in project automlops-sandbox 
Operation "operations/acat.p2-45373616427-53cf77d8-4efc-400c-80f5-ba4b73a09099" finished successfully.
 Checking for Artifact Registry: vertex-mlops-af in project automlops-sandbox 
Listing items under project automlops-sandbox, location us-central1.

vertex-mlops-af  DOCKER  STANDARD_REPOSITORY  Artifact Registry vertex-mlops-af in us-central1.  us-central1          Google-managed key  2023-01-11T22:12:26  2023-02-03T17:06:10  9945.322
Artifact Registry: vertex-mlops-af already exists in project automlops-sandbox
 Checking for GS Bucket: automlops-sandbox-bucket in project automlops-sandbox 
gs://automlops-sandbox-bucket/
GS Bucket: automlops-sandbox-bucket already exists in project automlops-sandbox
 Checking for Service Account: vertex-pipelines in project automlops-sandbox 
Pipeline Runner Service Account         vertex-pip

## 2. AutoMLOps Pipeline - Kubeflow Spec
This workflow will generate a pipeline using Kubeflow spec. `generate()` will create all the necessary files but not run them. `go()` will create all the necessary files, resources, push the code to the source repo to trigger the build, and then submit a Pipeline training job to Vertex AI. Please see the [readme](https://github.com/GoogleCloudPlatform/automlops/blob/main/README.md) for more information.

**Note: This workflow requires python packages `kfp` and `google-cloud-aiplatform`.**

## Imports

In [15]:
import json
import pandas as pd
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline, component, Artifact, Dataset, Input, Metrics, Model, Output, InputPath, OutputPath
from google.cloud import aiplatform
from google.cloud import aiplatform_v1
import datetime

from AutoMLOps import AutoMLOps

## Data Loading
Define a Kubeflow custom component for loading and creating a dataset. You must specify the `output_component_file` with the name of your component. For `AutoMLOps` to know where to find the Kubeflow component spec, set this variable to the following string `f"{AutoMLOps.OUTPUT_DIR}/your_component_name.yaml"`

In [16]:
@component(
    packages_to_install=[
        "google-cloud-bigquery", 
        "pandas",
        "pyarrow",
        "db_dtypes"
    ],
    base_image="python:3.9",
    output_component_file=f"{AutoMLOps.OUTPUT_DIR}/create_dataset.yaml"
)
def create_dataset(
    bq_table: str,
    output_data_path: OutputPath("Dataset"),
    project: str
):
    from google.cloud import bigquery
    import pandas as pd
    bq_client = bigquery.Client(project=project)


    def get_query(bq_input_table: str) -> str:
        """Generates BQ Query to read data.

        Args:
        bq_input_table: The full name of the bq input table to be read into
        the dataframe (e.g. <project>.<dataset>.<table>)
        Returns: A BQ query string.
        """
        return f"""
        SELECT *
        FROM `{bq_input_table}`
        """

    def load_bq_data(query: str, client: bigquery.Client) -> pd.DataFrame:
        """Loads data from bq into a Pandas Dataframe for EDA.
        Args:
        query: BQ Query to generate data.
        client: BQ Client used to execute query.
        Returns:
        pd.DataFrame: A dataframe with the requested data.
        """
        df = client.query(query).to_dataframe()
        return df

    dataframe = load_bq_data(get_query(bq_table), bq_client)
    dataframe.to_csv(output_data_path)

## Model Training
Define a Kubeflow custom component for training a model.

In [17]:
@component(
    packages_to_install=[
        "scikit-learn",
        "pandas",
        "joblib",
        "tensorflow"
    ],
    base_image="python:3.9",
    output_component_file=f"{AutoMLOps.OUTPUT_DIR}/train_model.yaml",
)
def train_model(
    output_model_directory: str,
    dataset: Input[Dataset],
    metrics: Output[Metrics],
    model: Output[Model]
):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_curve
    from sklearn.model_selection import train_test_split
    from joblib import dump
    import pandas as pd
    import tensorflow as tf
    import pickle
    import os
    
    def save_model(model, uri):
        """Saves a model to uri."""
        with tf.io.gfile.GFile(uri, 'w') as f:
            pickle.dump(model, f)
    
    df = pd.read_csv(dataset.path)
    labels = df.pop("Class").tolist()
    data = df.values.tolist()
    x_train, x_test, y_train, y_test = train_test_split(data, labels)
    skmodel = DecisionTreeClassifier()
    skmodel.fit(x_train,y_train)
    score = skmodel.score(x_test,y_test)
    print('accuracy is:',score)
    metrics.log_metric("accuracy",(score * 100.0))
    metrics.log_metric("framework", "Scikit Learn")
    metrics.log_metric("dataset_size", len(df))

    output_uri = os.path.join(output_model_directory, f'model.pkl')
    save_model(skmodel, output_uri)
    model.path = output_model_directory

## Uploading & Deploying the Model
Define a Kubeflow custom component for uploading and deploying a model in Vertex AI.

In [18]:
@component(
    packages_to_install=[
        "google-cloud-aiplatform"
    ],
    base_image="python:3.9",
    output_component_file=f"{AutoMLOps.OUTPUT_DIR}/deploy_model.yaml",
)
def deploy_model(
    model: Input[Model],
    project: str,
    region: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model]
):
    from google.cloud import aiplatform
    aiplatform.init(project=project, location=region)
    deployed_model = aiplatform.Model.upload(
        display_name="beans-model-pipeline",
        artifact_uri = model.uri,
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
    )
    endpoint = deployed_model.deploy(machine_type="n1-standard-4")
    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

## Define the Pipeline
When using Kubeflow spec, place the `%%define_kfp_pipeline` cell magic decorator at the top of your pipeline cell. This cell magic decorator comes built-in with AutoMLOps. Do not change the name of the function `def pipeline` - AutoMLOps expects the function name to be `pipeline`. Everything else is configurable. 

In [19]:
%%define_kfp_pipeline

@dsl.pipeline(name='training-pipeline')
def pipeline(bq_table: str,
             output_model_directory: str,
             project: str,
             region: str,
            ):

    dataset_task = create_dataset(
        bq_table=bq_table, 
        project=project)

    model_task = train_model(
        output_model_directory=output_model_directory,
        dataset=dataset_task.output)

    deploy_task = deploy_model(
        model=model_task.outputs["model"],
        project=project,
        region=region)


## Define the Pipeline Arguments

In [20]:
pipeline_params = {
    "bq_table": f"{PROJECT_ID}.test_dataset.dry-beans",
    "output_model_directory": f"gs://{PROJECT_ID}-bucket/trained_models/{datetime.datetime.now()}",
    "project": f"{PROJECT_ID}",
    "region": "us-central1"
}

## Generate and Run the pipeline
`AutoMLOps.generate` generates the code for the MLOps pipeline. `AutoMLOps.go` generates the code and runs the pipeline.

In [21]:
AutoMLOps.go(project_id=PROJECT_ID, pipeline_params=pipeline_params, use_kfp_spec=True, run_local=False, schedule_pattern='0 */12 * * *')

 Updating required API services in project automlops-sandbox 
Operation "operations/acat.p2-45373616427-d614c17e-4a53-466f-996d-ee650bc44464" finished successfully.
 Checking for Artifact Registry: vertex-mlops-af in project automlops-sandbox 
Listing items under project automlops-sandbox, location us-central1.

vertex-mlops-af  DOCKER  STANDARD_REPOSITORY  Artifact Registry vertex-mlops-af in us-central1.  us-central1          Google-managed key  2023-01-11T22:12:26  2023-03-06T18:23:43  12330.042
Artifact Registry: vertex-mlops-af already exists in project automlops-sandbox
 Checking for GS Bucket: automlops-sandbox-bucket in project automlops-sandbox 
gs://automlops-sandbox-bucket/
GS Bucket: automlops-sandbox-bucket already exists in project automlops-sandbox
 Checking for Service Account: vertex-pipelines in project automlops-sandbox 
Pipeline Runner Service Account         vertex-pipelines@automlops-sandbox.iam.gserviceaccount.com  False
Service Account: vertex-pipelines already 

INFO:root:Pushing code to automlops branch, triggering cloudbuild...
INFO:root:Cloudbuild job running at: https://console.cloud.google.com/cloud-build/builds;region=us-central1
INFO:root:
#################################################################
#                                                               #
#                       RESOURCES MANIFEST                      #
#---------------------------------------------------------------#
#     Generated resources can be found at the following urls    #
#                                                               #
#################################################################

INFO:root:Google Cloud Storage Bucket: https://console.cloud.google.com/storage/automlops-sandbox-bucket
INFO:root:Artifact Registry: https://console.cloud.google.com/artifacts/docker/automlops-sandbox/us-central1/vertex-mlops-af
INFO:root:Service Accounts: https://console.cloud.google.com/iam-admin/serviceaccounts?project=automlops-sandbox
INFO:r

To https://source.developers.google.com/p/automlops-sandbox/r/AutoMLOps-repo
   83b14ed..1200722  automlops -> automlops
